In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

KeyboardInterrupt: 

In [ ]:
!pip -q install ultralytics

In [ ]:
import os
import shutil
import cv2
import numpy as np
import hashlib
from pathlib import Path
import logging

# Configure logging to suppress per-image output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class DatasetCleaner:
    """Cleans the dataset by removing duplicate and corrupted images along with their labels."""
    
    def __init__(self, input_img_dir='/kaggle/input/detection-cars/images_cars', 
                 input_label_dir='/kaggle/input/detection-cars/labels_cars', 
                 output_dir='/kaggle/working/detection-cars-cleaned'):
        self.input_img_dir = Path(input_img_dir)
        self.input_label_dir = Path(input_label_dir)
        self.output_dir = Path(output_dir)
        self.output_img_dir = self.output_dir / 'images_cars'
        self.output_label_dir = self.output_dir / 'labels_cars'
        self.duplicates = []
        self.corrupted = []
        
    def copy_dataset(self):
        """Copy the dataset to the output directory."""
        logger.info("Copying dataset to output directory...")
        print("INFO: Copying dataset to output directory...")
        self.output_img_dir.mkdir(parents=True, exist_ok=True)
        self.output_label_dir.mkdir(parents=True, exist_ok=True)
        
        # Copy images
        for img_file in self.input_img_dir.glob('*.jpg'):
            shutil.copy(img_file, self.output_img_dir / img_file.name)
        
        # Copy labels
        for label_file in self.input_label_dir.glob('*.txt'):
            shutil.copy(label_file, self.output_label_dir / label_file.name)
        
        logger.info("Dataset copied successfully.")
        print("INFO: Dataset copied successfully.")
    
    def compute_image_hash(self, img_path):
        """Compute SHA256 hash of an image for duplicate detection."""
        try:
            with open(img_path, 'rb') as f:
                return hashlib.sha256(f.read()).hexdigest()
        except Exception:
            return None
    
    def is_valid_image(self, img_path):
        """Check if an image is valid and not corrupted."""
        try:
            img = cv2.imread(str(img_path))
            if img is None or np.all(img == 0) or np.all(img == 255):
                return False
            return True
        except Exception:
            return False
    
    def find_duplicates(self):
        """Find duplicate images based on content hash."""
        hashes = {}
        for img_file in self.output_img_dir.glob('*.jpg'):
            img_hash = self.compute_image_hash(img_file)
            if img_hash:
                if img_hash in hashes:
                    self.duplicates.append(img_file)
                else:
                    hashes[img_hash] = img_file
    
    def find_corrupted(self):
        """Find corrupted images."""
        for img_file in self.output_img_dir.glob('*.jpg'):
            if not self.is_valid_image(img_file):
                self.corrupted.append(img_file)
    
    def remove_files(self, files, file_type):
        """Remove images and their corresponding labels."""
        removed_count = 0
        for file in files:
            label_file = self.output_label_dir / f'{file.stem}.txt'
            try:
                if file.exists():
                    file.unlink()
                    removed_count += 1
                if label_file.exists():
                    label_file.unlink()
            except Exception as e:
                logger.warning(f"Failed to remove {file_type} file {file} or its label: {e}")
                print(f"WARNING: Failed to remove {file_type} file {file} or its label: {e}")
        return removed_count
    
    def clean(self):
        """Clean the dataset by removing duplicates and corrupted images."""
        # Step 1: Copy dataset
        self.copy_dataset()
        
        # Step 2: Find duplicates
        logger.info("Checking for duplicate images...")
        print("INFO: Checking for duplicate images...")
        self.find_duplicates()
        duplicate_count = len(self.duplicates)
        
        # Step 3: Find corrupted images
        logger.info("Checking for corrupted images...")
        print("INFO: Checking for corrupted images...")
        self.find_corrupted()
        corrupted_count = len(self.corrupted)
        
        # Step 4: Handle corrupted images
        removed_corrupted = 0
        if corrupted_count > 0:
            print(f"INFO: Found {corrupted_count} corrupted images.")
            response = input("Do you want to remove corrupted images and their labels? (y/n): ").strip().lower()
            if response == 'y':
                removed_corrupted = self.remove_files(self.corrupted, "corrupted")
            else:
                print("INFO: Corrupted images will not be removed.")
        
        # Step 5: Remove duplicates
        removed_duplicates = self.remove_files(self.duplicates, "duplicate")
        
        # Step 6: Generate report
        total_images = len(list(self.output_img_dir.glob('*.jpg')))
        total_labels = len(list(self.output_label_dir.glob('*.txt')))
        report = (
            f"Cleaning Report:\n"
            f"- Total images after cleaning: {total_images}\n"
            f"- Total labels after cleaning: {total_labels}\n"
            f"- Duplicate images found and removed: {removed_duplicates}\n"
            f"- Corrupted images found: {corrupted_count}\n"
            f"- Corrupted images removed: {removed_corrupted}\n"
        )
        logger.info(report)
        print(f"INFO: {report}")
        
        # Save report to file
        with open(self.output_dir / 'cleaning_report.txt', 'w') as f:
            f.write(report)
        print(f"INFO: Report saved to {self.output_dir / 'cleaning_report.txt'}")

def main():
    cleaner = DatasetCleaner()
    cleaner.clean()

if __name__ == "__main__":
    main()

In [ ]:
from pathlib import Path
import logging
import osw
import glob
import shutil

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def check_labels(label_dir='/kaggle/input/detection-cars/labels_cars',
                 image_dir='/kaggle/input/detection-cars/images_cars',
                 output_dir='/kaggle/working/detection-cars-cleaned'):
    """Check label files for valid class IDs (0-3), copy valid files to output_dir, remove invalid labels and corresponding images."""
    label_dir = Path(label_dir)
    image_dir = Path(image_dir)
    output_dir = Path(output_dir)
    
    # Check if input directories exist
    if not label_dir.exists():
        logging.error(f"Label directory {label_dir} does not exist")
        print(f"ERROR: Label directory {label_dir} does not exist")
        return
    if not image_dir.exists():
        logging.error(f"Image directory {image_dir} does not exist")
        print(f"ERROR: Image directory {image_dir} does not exist")
        return
    
    # Create output directories
    output_image_dir = output_dir / 'images'
    output_label_dir = output_dir / 'labels'
    output_image_dir.mkdir(parents=True, exist_ok=True)
    output_label_dir.mkdir(parents=True, exist_ok=True)
    
    valid_class_ids = {0, 1, 2, 3}  # car, truck, bus, motorcycle
    invalid_labels = []
    valid_labels = 0
    copied_images = 0
    
    # Supported image extensions
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    
    # Check each label file
    for label_file in label_dir.glob('*.txt'):
        try:
            with open(label_file, 'r') as f:
                lines = f.readlines()
            
            if not lines:
                invalid_labels.append((label_file, "Empty file"))
                continue
            
            valid = True
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    invalid_labels.append((label_file, f"Wrong number of columns ({len(parts)})"))
                    valid = False
                    break
                try:
                    class_id = int(parts[0])
                    if class_id not in valid_class_ids:
                        invalid_labels.append((label_file, f"Invalid class ID ({class_id})"))
                        valid = False
                        break
                    # Verify coordinates are valid floats
                    for j in range(1, 5):
                        float(parts[j])
                except ValueError:
                    invalid_labels.append((label_file, f"Invalid number format in line: {line.strip()}"))
                    valid = False
                    break
            
            if valid:
                # Copy valid label to output directory
                shutil.copy(label_file, output_label_dir / label_file.name)
                valid_labels += 1
                # Copy corresponding image (any supported extension)
                img_found = False
                for ext in image_extensions:
                    img_file = image_dir / f'{label_file.stem}{ext}'
                    if img_file.exists():
                        shutil.copy(img_file, output_image_dir / img_file.name)
                        copied_images += 1
                        img_found = True
                        break
                if not img_found:
                    logging.warning(f"No image found for valid label {label_file.name}")
                    # Remove the copied label if no image is found
                    os.remove(output_label_dir / label_file.name)
                    valid_labels -= 1
            else:
                # Log invalid label, don't copy
                img_found = False
                for ext in image_extensions:
                    img_file = image_dir / f'{label_file.stem}{ext}'
                    if img_file.exists():
                        img_found = True
                        break
                if not img_found:
                    logging.warning(f"No image found for invalid label {label_file.name}")
        except Exception as e:
            invalid_labels.append((label_file, f"Failed to process: {e}"))
            img_found = False
            for ext in image_extensions:
                img_file = image_dir / f'{label_file.stem}{ext}'
                if img_file.exists():
                    img_found = True
                    break
            if not img_found:
                logging.warning(f"No image found for invalid label {label_file.name}")
    
    # Generate report
    report = (
        f"Label Check Report:\n"
        f"- Total labels checked: {valid_labels + len(invalid_labels)}\n"
        f"- Valid labels copied: {valid_labels}\n"
        f"- Invalid labels skipped: {len(invalid_labels)}\n"
        f"- Images copied: {copied_images}\n"
    )
    if invalid_labels:
        report += f"- Invalid labels (first 5):\n"
        for label_file, reason in invalid_labels[:5]:
            report += f"  - {label_file.name}: {reason}\n"
    
    logging.info(report)
    print(f"INFO: {report}")
    
    # Save report to /kaggle/working/
    report_path = Path('/kaggle/working/label_check_report.txt')
    try:
        with open(report_path, 'w') as f:
            f.write(report)
        print(f"INFO: Report saved to {report_path}")
    except Exception as e:
        logging.error(f"Failed to save report: {e}")
        print(f"ERROR: Failed to save report: {e}")
    
if __name__ == "__main__":
    check_labels()

In [ ]:
# !pip install osw

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from shutil import copyfile
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DatasetAugmenter:
    def __init__(self, image_dir, label_dir, output_image_dir, output_label_dir, image_exts=['.jpg', '.jpeg', '.png', '.bmp'], augmentations_per_image=1):
        """Initialize the augmenter with directories and settings."""
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.output_image_dir = output_image_dir
        self.output_label_dir = output_label_dir
        self.image_exts = image_exts
        self.augmentations_per_image = augmentations_per_image
        self.augmented_images = 0
        self.augmented_labels = 0
        self.valid_class_ids = {0, 1, 2, 3}  # car, truck, bus, motorcycle

        # Create output directories
        os.makedirs(self.output_image_dir, exist_ok=True)
        os.makedirs(self.output_label_dir, exist_ok=True)

        # Define augmentation pipeline matching aug_hyp.yaml
        self.transform = A.Compose([
            A.HorizontalFlip(p=0.5),  # Matches fliplr: 0.5
            A.VerticalFlip(p=0.5),    # Matches flipud: 0.5
            A.Rotate(limit=10, p=0.5),  # Matches degrees: 10.0
            A.HueSaturationValue(hue_shift_limit=0.015*360, sat_shift_limit=0.7*100, val_shift_limit=0.4*100, p=0.5),  # Matches hsv_h, hsv_s, hsv_v
            A.Affine(translate_percent=0.1, scale=(0.5, 1.5), shear=2.0, p=0.5),  # Matches translate, scale, shear
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
            A.GaussNoise(p=0.2),
        ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.3))

    def augment_image(self, image, bboxes, class_labels):
        """Apply augmentation to a single image and its labels."""
        try:
            augmented = self.transform(image=image, bboxes=bboxes, class_labels=class_labels)
            return augmented['image'], augmented['bboxes'], augmented['class_labels']
        except Exception as e:
            logging.warning(f"Augmentation failed: {e}")
            return None, None, None

    def process_dataset(self):
        """Process all images and labels to create augmented dataset."""
        for image_file in os.listdir(self.image_dir):
            # Check if file has a supported extension
            if any(image_file.lower().endswith(ext) for ext in self.image_exts):
                image_path = os.path.join(self.image_dir, image_file)
                label_path = os.path.join(self.label_dir, image_file.rsplit('.', 1)[0] + '.txt')

                if not os.path.exists(label_path):
                    logging.warning(f"No label found for {image_file}, skipping")
                    continue

                # Read image
                image = cv2.imread(image_path)
                if image is None:
                    logging.warning(f"Failed to read image {image_file}, skipping")
                    continue
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Read labels
                bboxes = []
                class_labels = []
                valid_label = True
                try:
                    with open(label_path, 'r') as f:
                        lines = f.readlines()
                        for line in lines:
                            parts = line.strip().split()
                            if len(parts) != 5:
                                valid_label = False
                                break
                            try:
                                class_id = int(parts[0])
                                if class_id not in self.valid_class_ids:
                                    valid_label = False
                                    break
                                x_center, y_center, width, height = map(float, parts[1:])
                                bboxes.append([x_center, y_center, width, height])
                                class_labels.append(class_id)
                            except ValueError:
                                valid_label = False
                                break
                except Exception as e:
                    valid_label = False
                    logging.warning(f"Failed to read label {label_path}: {e}")

                if not valid_label:
                    logging.warning(f"Skipping {image_file}: Invalid label format or class ID")
                    continue

                # Copy original image and label
                copyfile(image_path, os.path.join(self.output_image_dir, image_file))
                copyfile(label_path, os.path.join(self.output_label_dir, image_file.rsplit('.', 1)[0] + '.txt'))

                # Create augmented versions
                for i in range(self.augmentations_per_image):
                    aug_image, aug_bboxes, aug_class_labels = self.augment_image(image, bboxes, class_labels)
                    if aug_image is not None:
                        # Save augmented image
                        aug_image_path = os.path.join(self.output_image_dir, f"aug_{i}_{image_file}")
                        cv2.imwrite(aug_image_path, cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))
                        self.augmented_images += 1

                        # Save augmented labels
                        aug_label_path = os.path.join(self.output_label_dir, f"aug_{i}_{image_file.rsplit('.', 1)[0]}.txt")
                        with open(aug_label_path, 'w') as f:
                            for class_id, bbox in zip(aug_class_labels, aug_bboxes):
                                x_center, y_center, width, height = bbox
                                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
                        self.augmented_labels += 1

    def get_report(self):
        """Return a report of the augmentation results."""
        total_images = len([f for f in os.listdir(self.output_image_dir) if any(f.lower().endswith(ext) for ext in self.image_exts)])
        total_labels = len([f for f in os.listdir(self.output_label_dir) if f.endswith('.txt')])
        return (
            f"Augmentation Report:\n"
            f"- Created {self.augmented_images} augmented image files.\n"
            f"- Created {self.augmented_labels} augmented label files.\n"
            f"- Total files in augmented dataset: {total_images} images and {total_labels} labels."
        )

# Usage
if __name__ == "__main__":
    augmenter = DatasetAugmenter(
        image_dir='/kaggle/working/detection-cars-cleaned/images',
        label_dir='/kaggle/working/detection-cars-cleaned/labels',
        output_image_dir='/kaggle/working/detection-cars-cleaned/augmented_images',
        output_label_dir='/kaggle/working/detection-cars-cleaned/augmented_labels',
        image_exts=['.jpg', '.jpeg', '.png', '.bmp'],
        augmentations_per_image=1
    )
    augmenter.process_dataset()
    print(augmenter.get_report())

In [ ]:
from pathlib import Path
import os
import random
import shutil
import logging
import yaml

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def split_data(image_dir='/kaggle/working/detection-cars-cleaned/augmented_images',
               label_dir='/kaggle/working/detection-cars-cleaned/augmented_labels',
               output_dir='/kaggle/working/detection-cars-cleaned',
               train_ratio=0.7, val_ratio=0.15, test_ratio=0.15,
               image_exts=['.jpg', '.jpeg', '.png', '.bmp']):
    """Split images and labels into train, val, test sets by moving files and create data.yaml."""
    image_dir = Path(image_dir)
    label_dir = Path(label_dir)
    output_dir = Path(output_dir)
    
    # Validate ratios
    if abs(train_ratio + val_ratio + test_ratio - 1.0) > 0.01:
        raise ValueError("Train, val, and test ratios must sum to 1.0")
    
    # Check if input directories exist
    if not image_dir.exists():
        logging.error(f"Image directory {image_dir} does not exist")
        print(f"ERROR: Image directory {image_dir} does not exist")
        return
    if not label_dir.exists():
        logging.error(f"Label directory {label_dir} does not exist")
        print(f"ERROR: Label directory {label_dir} does not exist")
        return
    
    # Create output directories
    for split in ['train', 'val', 'test']:
        os.makedirs(output_dir / split / 'images', exist_ok=True)
        os.makedirs(output_dir / split / 'labels', exist_ok=True)
    
    # Get list of images and ensure corresponding labels exist
    image_files = [f for f in os.listdir(image_dir) if any(f.lower().endswith(ext) for ext in image_exts)]
    valid_pairs = []
    for img_file in image_files:
        label_file = img_file.rsplit('.', 1)[0] + '.txt'
        if os.path.exists(label_dir / label_file):
            valid_pairs.append((img_file, label_file))
        else:
            logging.warning(f"No label found for {img_file}, skipping")
    
    # Shuffle and split
    random.shuffle(valid_pairs)
    total_pairs = len(valid_pairs)
    train_end = int(train_ratio * total_pairs)
    val_end = train_end + int(val_ratio * total_pairs)
    
    train_pairs = valid_pairs[:train_end]
    val_pairs = valid_pairs[train_end:val_end]
    test_pairs = valid_pairs[val_end:]
    
    # Move files to respective directories
    def move_files(pairs, split):
        img_count = 0
        label_count = 0
        for img_file, label_file in pairs:
            try:
                shutil.move(image_dir / img_file, output_dir / split / 'images' / img_file)
                shutil.move(label_dir / label_file, output_dir / split / 'labels' / label_file)
                img_count += 1
                label_count += 1
            except Exception as e:
                logging.warning(f"Failed to move {img_file} or {label_file}: {e}")
        return img_count, label_count
    
    # Move files for each split
    train_img_count, train_label_count = move_files(train_pairs, 'train')
    val_img_count, val_label_count = move_files(val_pairs, 'val')
    test_img_count, test_label_count = move_files(test_pairs, 'test')
    
    # Create data.yaml
    data_yaml = {
        'train': str(output_dir / 'train' / 'images'),
        'val': str(output_dir / 'val' / 'images'),
        'test': str(output_dir / 'test' / 'images'),
        'nc': 4,
        'names': ['car', 'truck', 'bus', 'motorcycle']
    }
    
    yaml_path = output_dir / 'data.yaml'
    try:
        with open(yaml_path, 'w') as f:
            yaml.safe_dump(data_yaml, f, sort_keys=False)
        logging.info(f"data.yaml created at {yaml_path}")
        print(f"INFO: data.yaml created at {yaml_path}")
    except Exception as e:
        logging.error(f"Failed to create data.yaml: {e}")
        print(f"ERROR: Failed to create data.yaml: {e}")
        raise
    
    # Clean up old directories
    try:
        if image_dir.exists() and not any(image_dir.iterdir()):
            shutil.rmtree(image_dir)
            logging.info(f"Removed empty directory {image_dir}")
        if label_dir.exists() and not any(label_dir.iterdir()):
            shutil.rmtree(label_dir)
            logging.info(f"Removed empty directory {label_dir}")
    except Exception as e:
        logging.warning(f"Failed to clean up old directories: {e}")
    
    # Generate report
    report = (
        f"Data Split Report:\n"
        f"- Total valid image-label pairs: {total_pairs}\n"
        f"- Train: {train_img_count} images, {train_label_count} labels\n"
        f"- Val: {val_img_count} images, {val_label_count} labels\n"
        f"- Test: {test_img_count} images, {test_label_count} labels\n"
        f"- data.yaml saved at: {yaml_path}"
    )
    logging.info(report)
    print(f"INFO: {report}")
    
    with open(output_dir / 'data_split_report.txt', 'w') as f:
        f.write(report)
    print(f"INFO: Report saved to {output_dir / 'data_split_report.txt'}")

if __name__ == "__main__":
    split_data()

In [ ]:
import yaml
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_aug_hyp_yaml(output_path='/kaggle/working/aug_hyp.yaml'):
    """Create aug_hyp.yaml with training and augmentation hyperparameters."""
    aug_hyp = {
        'optimizer': 'AdamW',
        'lr0': 0.001,
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.005,
        'warmup_epochs': 3.0,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 7.5,
        'cls': 1.0,
        'dfl': 1.5,
        'kobj': 1.0,
        'iou': 0.5,
        'conf': 0.25,
        'hsv_h': 0.015,
        'hsv_s': 0.7,
        'hsv_v': 0.4,
        'degrees': 10.0,
        'translate': 0.1,
        'scale': 0.5,
        'shear': 2.0,
        'perspective': 0.0,
        'flipud': 0.5,
        'fliplr': 0.5,
        'mosaic': 0.5,
        'mixup': 0.2,
        'copy_paste': 0.0,
        'auto_augment': 'none',
        'erasing': 0.0
    }
    
    try:
        with open(output_path, 'w') as f:
            yaml.safe_dump(aug_hyp, f, sort_keys=False)
        logging.info(f"aug_hyp.yaml created at {output_path}")
        print(f"INFO: aug_hyp.yaml created at {output_path}")
        
        # Verify file exists
        if Path(output_path).exists():
            with open(output_path, 'r') as f:
                print(f"INFO: Contents of aug_hyp.yaml:\n{f.read()}")
        else:
            raise FileNotFoundError(f"aug_hyp.yaml was not created at {output_path}")
    except Exception as e:
        logging.error(f"Failed to create aug_hyp.yaml: {e}")
        print(f"ERROR: Failed to create aug_hyp.yaml: {e}")
        raise

if __name__ == "__main__":
    create_aug_hyp_yaml()

In [ ]:
# ===================== Training Note =====================
# Phase 1: Trained from scratch using yolov8l.pt for 33 epochs
# Phase 2: Resumed training from saved checkpoint (best.pt) for 17 additional epochs
# Total training: 50 epochs
# ========================================================

In [ ]:
# Phase 1

In [ ]:
import os
import logging
from ultralytics import YOLO
import yaml
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class YOLOv8Trainer:
    """Class to handle YOLOv8l training with fine-tuning, early stopping, and checkpoint cleanup."""
    
    def __init__(self,
                 data_yaml='/kaggle/working/detection-cars-cleaned/data.yaml',
                 hyp_yaml='/kaggle/working/aug_hyp.yaml',
                 output_dir='/kaggle/working/runs',
                 epochs=50,
                 batch_size=16,
                 imgsz=640,
                 patience=20):
        """Initialize trainer with training configurations."""
        self.data_yaml = Path(data_yaml)
        self.hyp_yaml = Path(hyp_yaml)
        self.output_dir = Path(output_dir)
        self.epochs = epochs
        self.batch_size = batch_size
        self.imgsz = imgsz
        self.patience = patience
        self.model = None
        self.hyp = None
        self.weights_dir = None
        
    def clean_old_checkpoints(self):
        """Remove old checkpoints, keep only best.pt and last.pt."""
        for checkpoint in self.weights_dir.glob("epoch_*.pt"):
            try:
                checkpoint.unlink()
                logging.info(f"Removed checkpoint: {checkpoint}")
            except Exception as e:
                logging.warning(f"Failed to remove {checkpoint}: {e}")
    
    def load_hyp_yaml(self):
        """Load hyperparameters from YAML file."""
        try:
            with open(self.hyp_yaml, 'r') as f:
                self.hyp = yaml.safe_load(f)
            logging.info(f"Loaded hyperparameters from {self.hyp_yaml}")
        except Exception as e:
            logging.error(f"Failed to load {self.hyp_yaml}: {e}")
            raise
    
    def validate_files(self):
        """Check if data.yaml and hyp.yaml exist."""
        if not self.data_yaml.exists():
            logging.error(f"Data YAML file {self.data_yaml} does not exist")
            return False
        if not self.hyp_yaml.exists():
            logging.error(f"Hyp YAML file {self.hyp_yaml} does not exist")
            return False
        return True
    
    def setup(self):
        """Setup training: validate files, load hyperparameters, create output dir."""
        if not self.validate_files():
            return False
        self.load_hyp_yaml()
        os.makedirs(self.output_dir, exist_ok=True)
        logging.info("Training setup completed")
        return True
    
    def train(self):
        """Train YOLOv8l model with fine-tuning, validate every 10 epochs."""
        try:
            self.model = YOLO('yolov8l.pt')  # Load pre-trained YOLOv8l
            epoch_step = 10  # Validate every 10 epochs
            current_epoch = 0
            
            while current_epoch < self.epochs:
                # Calculate remaining epochs for this step
                epochs_to_run = min(epoch_step, self.epochs - current_epoch)
                
                # Train for epochs_to_run
                results = self.model.train(
                    data=str(self.data_yaml),
                    epochs=epochs_to_run,
                    batch=self.batch_size,
                    imgsz=self.imgsz,
                    device=0,
                    project=str(self.output_dir),
                    name='yolov8l_vehicles',
                    exist_ok=True,
                    save=True,
                    save_period=-1,
                    plots=True,
                    verbose=True,
                    patience=self.patience,
                    resume=current_epoch > 0,  # Resume from previous checkpoint if not first step
                    **self.hyp
                )
                
                # Update current epoch
                current_epoch += epochs_to_run
                
                # Run validation manually after every epoch_step
                if current_epoch % epoch_step == 0:  # Don't validate after final epoch
                    val_metrics = self.model.val(data=str(self.data_yaml), split='val')
                    logging.info(f"Validation Metrics at epoch {current_epoch}: {val_metrics}")
                    print(f"INFO: Validation Metrics at epoch {current_epoch}: {val_metrics}")
                
                # Clean old checkpoints
                self.weights_dir = self.output_dir / 'yolov8l_vehicles' / 'weights'
                self.clean_old_checkpoints()
            
            logging.info("Training completed")
            return results
        except Exception as e:
            logging.error(f"Training failed: {e}")
            raise
    
    def evaluate(self):
        """Evaluate model on test set."""
        if self.model is None:
            logging.error("No model for evaluation")
            return
        try:
            metrics = self.model.val(data=str(self.data_yaml), split='test')
            logging.info(f"Test Metrics: {metrics}")
            print(f"INFO: Test Metrics: {metrics}")
            return metrics
        except Exception as e:
            logging.error(f"Evaluation failed: {e}")
            raise
    
    def save_final_model(self):
        """Save final model as final.pt."""
        if self.model is None:
            logging.error("No model to save")
            return
        try:
            final_path = self.weights_dir / 'final.pt'
            self.model.save(str(final_path))
            logging.info(f"Final model saved to {final_path}")
            print(f"INFO: Final model saved to {final_path}")
            self.clean_old_checkpoints()
        except Exception as e:
            logging.error(f"Failed to save final model: {e}")
            raise
    
    def run(self):
        """Run full training pipeline."""
        if not self.setup():
            return
        self.train()
        self.evaluate()
        self.save_final_model()

if __name__ == "__main__":
    trainer = YOLOv8Trainer()
    trainer.run()

In [ ]:
# Phase 2

In [ ]:
import os
import logging
import torch
import torch.nn as nn
from ultralytics import YOLO
import yaml
from pathlib import Path
from datetime import datetime
import shutil
import numpy as np
from torch.utils.data import WeightedRandomSampler
import albumentations as A
import cv2

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True,
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('/kaggle/working/logs.txt')
    ]
)

class YOLOv8Trainer:
    def __init__(self,
                 data_yaml='/kaggle/working/detection-cars-cleaned/data.yaml',
                 hyp_yaml='/kaggle/working/aug_hyp.yaml',
                 output_dir='/kaggle/working/runs',
                 epochs=30,
                 batch_size=16,
                 imgsz=640,
                 patience=20,
                 save_period=1):
        print("Initializing YOLOv8Trainer")
        logging.info("Initializing YOLOv8Trainer")
        
        self.data_yaml = Path(data_yaml)
        self.hyp_yaml = Path(hyp_yaml)
        self.output_dir = Path(output_dir)
        self.epochs = epochs
        self.batch_size = batch_size
        self.imgsz = imgsz
        self.patience = patience
        self.save_period = save_period
        self.model = None
        self.hyp = None
        self.weights_dir = None
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        logging.info(f"Using device: {self.device}")
        self.checkpoint_input_path = Path('/kaggle/input/mostafachekpoints/pytorch/default/1/mostafacheckpoints.pt') 
        self.checkpoint_output_path = None
        self.best_score = 0.0
        self.best_checkpoint_score = 0.0
        self.best_recall = 0.0
        self.no_improve_epochs = 0
        self.current_epoch = 0

    def validate_files(self):
        """Check if data.yaml and hyp.yaml exist."""
        print("Entering validate_files")
        logging.info("Entering validate_files")
        
        valid = True
        if not self.data_yaml.exists():
            logging.error(f"Data YAML file {self.data_yaml} does not exist")
            print(f"ERROR: Data YAML file {self.data_yaml} does not exist")
            valid = False
        if not self.hyp_yaml.exists():
            logging.error(f"Hyp YAML file {self.hyp_yaml} does not exist")
            print(f"ERROR: Hyp YAML file {self.hyp_yaml} does not exist")
            valid = False
        return valid

    def load_hyp_yaml(self):
        """Load hyperparameters from YAML file."""
        print("Entering load_hyp_yaml")
        logging.info("Entering load_hyp_yaml")
        
        if not self.hyp_yaml.exists():
            logging.error(f"Hyperparameter file {self.hyp_yaml} not found.")
            print(f"ERROR: Hyperparameter file {self.hyp_yaml} not found.")
            raise FileNotFoundError(f"Hyperparameter file {self.hyp_yaml} not found.")
        try:
            with open(self.hyp_yaml, 'r') as f:
                self.hyp = yaml.safe_load(f)
            self.hyp['cls'] = self.hyp.get('cls', 0.5) * 1.5
            logging.info(f"Loaded hyperparameters from {self.hyp_yaml}")
            print(f"Loaded hyperparameters from {self.hyp_yaml}")
            if not isinstance(self.hyp, dict):
                logging.error(f"Hyperparameters loaded from {self.hyp_yaml} are not a dictionary.")
                print(f"ERROR: Hyperparameters loaded from {self.hyp_yaml} are not a dictionary.")
                raise ValueError("Invalid hyperparameter format")
        except yaml.YAMLError as e:
            logging.error(f"Error parsing YAML file {self.hyp_yaml}: {e}")
            print(f"ERROR: Error parsing YAML file {self.hyp_yaml}: {e}")
            raise ValueError(f"Error parsing YAML file {self.hyp_yaml}") from e

    def load_initial_model(self):
        """Load pretrained yolov8l.pt, update head for nc=4, then load checkpoint state dict without freeze."""
        print("Entering load_initial_model")
        logging.info("Entering load_initial_model")
        
        try:
            # Validate data.yaml
            with open(self.data_yaml, 'r') as f:
                data_config = yaml.safe_load(f)
            if data_config['nc'] != 4:
                raise ValueError(f"Expected 4 classes in {self.data_yaml}, found {data_config['nc']}")
            expected_names = ['car', 'truck', 'bus', 'motorcycle']
            if data_config['names'] != expected_names:
                raise ValueError(f"Expected class names {expected_names} in {self.data_yaml}, found {data_config['names']}")
            
            # Load pretrained yolov8l.pt
            self.model = YOLO('yolov8l.pt')
            self.model.model.yaml['nc'] = 4
            
            # No freeze: all layers are trainable
            for param in self.model.model.parameters():
                param.requires_grad = True
            
            # Update head for nc=4
            head_module = self.model.model.model[22]
            for i in range(3):
                conv_layer = head_module.cv3[i][2]
                if conv_layer.out_channels == 80:
                    new_conv = nn.Conv2d(
                        in_channels=conv_layer.in_channels,
                        out_channels=4,
                        kernel_size=conv_layer.kernel_size,
                        stride=conv_layer.stride,
                        padding=conv_layer.padding,
                        bias=conv_layer.bias is not None
                    ).to(self.device)
                    head_module.cv3[i][2] = new_conv
            
            # Load checkpoint state dict
            checkpoint_path = str(self.checkpoint_input_path)
            if not Path(checkpoint_path).exists():
                logging.error(f"Checkpoint file {checkpoint_path} not found.")
                print(f"ERROR: Checkpoint file {checkpoint_path} not found.")
                raise FileNotFoundError(f"Checkpoint file {checkpoint_path} not found.")
            
            checkpoint = torch.load(checkpoint_path, map_location=self.device)
            self.model.model.load_state_dict(checkpoint['model_state_dict'])
            
            # Verify parameter count after loading
            total_params = sum(p.numel() for p in self.model.model.parameters() if p.requires_grad)
            logging.info(f"Total trainable parameters after loading: {total_params}")
            print(f"Total trainable parameters after loading: {total_params}")
            
            # Log detailed parameter groups for debugging
            bn_weights = [name for name, param in self.model.model.named_parameters() if 'bn' in name and '.weight' in name]
            bn_biases = [name for name, param in self.model.model.named_parameters() if 'bn' in name and '.bias' in name]
            other_weights = [name for name, param in self.model.model.named_parameters() if '.weight' in name and 'bn' not in name]
            other_biases = [name for name, param in self.model.model.named_parameters() if '.bias' in name and 'bn' not in name]
            logging.info(f"BatchNorm weights count: {len(bn_weights)}")
            logging.info(f"BatchNorm biases count: {len(bn_biases)}")
            logging.info(f"Other weights count: {len(other_weights)}")
            logging.info(f"Other biases count: {len(other_biases)}")
            print(f"BatchNorm weights count: {len(bn_weights)}")
            print(f"BatchNorm biases count: {len(bn_biases)}")
            print(f"Other weights count: {len(other_weights)}")
            print(f"Other biases count: {len(other_biases)}")
            
            logging.info(f"Loaded pretrained yolov8l.pt with nc=4 and checkpoint {checkpoint_path} (no freeze)")
            print(f"Loaded pretrained yolov8l.pt with nc=4 and checkpoint {checkpoint_path} (no freeze)")
        except Exception as error:
            logging.critical(f"Failed to load model or checkpoint: {str(error)}")
            print(f"CRITICAL: Failed to load model or checkpoint: {str(error)}")
            raise RuntimeError("Could not load model or checkpoint.") from error

    def setup(self):
        """Setup training: validate files, load hyperparameters, create output dir."""
        print("Entering setup")
        logging.info("Entering setup")
        
        if not self.validate_files():
            logging.error("Setup failed: Required files missing.")
            print("ERROR: Setup failed: Required files missing.")
            return False
        
        try:
            self.load_hyp_yaml()
            os.makedirs(self.output_dir, exist_ok=True)
            self.load_initial_model()
            if self.model is None:
                logging.error("Setup failed: Model not initialized.")
                print("ERROR: Setup failed: Model not initialized.")
                return False
            self.weights_dir = self.output_dir / 'yolov8l_vehicles_custom_weights'
            os.makedirs(self.weights_dir, exist_ok=True)
            logging.info("Training setup completed")
            print("Training setup completed")
            return True
        except Exception as error:
            logging.critical(f"Setup failed: {str(error)}")
            print(f"CRITICAL: Setup failed: {str(error)}")
            return False

    def clean_old_checkpoints(self):
        """Remove old checkpoints except required files."""
        print("Entering clean_old_checkpoints")
        logging.info("Entering clean_old_checkpoints")
        
        if not self.weights_dir:
            logging.warning("Weights directory not set. Skipping checkpoint cleaning.")
            print("WARNING: Weights directory not set. Skipping checkpoint cleaning.")
            return 
        keep_files = ['finalchechpoints.pt', 'bestmodel.pt', 'lastfinal.pt', 'mostafafinal.pt']
        checkpoint_dir = self.weights_dir / 'check_points1'
        try:
            if checkpoint_dir.exists():
                for checkpoint in checkpoint_dir.glob("*.pt"):
                    if checkpoint.name not in keep_files:
                        checkpoint.unlink()
                        logging.info(f"Removed old checkpoint: {checkpoint}")
                        print(f"Removed old checkpoint: {checkpoint}")
        except Exception as e:
            logging.error(f"Error cleaning checkpoints in {checkpoint_dir}: {e}")
            print(f"ERROR: Error cleaning checkpoints in {checkpoint_dir}: {e}")

    def save_checkpoint(self, epoch, map50_95, recall):
        """Save checkpoint with model weights, optimizer state, epoch, loss, and metrics."""
        print(f"Entering save_checkpoint for epoch {epoch + 1}/{self.epochs}")
        logging.info(f"Entering save_checkpoint for epoch {epoch + 1}/{self.epochs}")
        
        if not self.weights_dir:
            logging.error("Cannot save checkpoint: Weights directory not set.")
            print("ERROR: Cannot save checkpoint: Weights directory not set.")
            return
            
        checkpoint_dir = self.weights_dir / 'check_points1'
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"Created/Verified checkpoint directory: {checkpoint_dir}")
        logging.info(f"Created/Verified checkpoint directory: {checkpoint_dir}")
        
        checkpoint_path = checkpoint_dir / "finalchechpoints.pt"
        last_path = checkpoint_dir / "lastfinal.pt"
        best_path = checkpoint_dir / "bestmodel.pt"
        self.checkpoint_output_path = checkpoint_path
        
        timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
        
        if self.model is None or self.model.model is None:
            logging.error("Model not initialized. Cannot save state dict.")
            print("ERROR: Model not initialized. Cannot save state dict.")
            return
            
        optimizer_state_dict = None
        if self.model.trainer and self.model.trainer.optimizer:
            optimizer_state_dict = self.model.trainer.optimizer.state_dict()
        else:
            logging.warning("Optimizer not available. Saving checkpoint without optimizer state.")
            print("WARNING: Optimizer not available. Saving checkpoint without optimizer state.")

        train_loss = getattr(self.model.trainer, 'loss', 0.0) if self.model.trainer else 0.0
        val_loss = getattr(self.model.trainer, 'val_loss', 0.0) if self.model.trainer else 0.0
        loss_diff = abs(train_loss - val_loss)
        stability_penalty = min(max(0, (loss_diff - 0.1) * 0.05), 1.0)  # Cap penalty at 1.0

        score = 0.5 * map50_95 + 0.5 * recall - stability_penalty

        try:
            checkpoint_data = {
                'epoch': epoch,
                'model_state_dict': self.model.model.state_dict(),
                'optimizer_state_dict': optimizer_state_dict,
                'loss': train_loss,
                'map50_95': map50_95,
                'recall': recall,
                'score': score,
                'timestamp': timestamp,
                'best_score_so_far': self.best_score,
                'best_recall': self.best_recall,
                'no_improve_epochs': self.no_improve_epochs
            }
            
            torch.save(checkpoint_data, checkpoint_path)
            logging.info(f"Saved checkpoint: {checkpoint_path} (Epoch: {epoch + 1}/{self.epochs}, Score: {score:.4f}, mAP@50:95: {map50_95:.4f}, Recall: {recall:.4f})")
            print(f"Saved checkpoint: {checkpoint_path} (Epoch: {epoch + 1}/{self.epochs}, Score: {score:.4f}, mAP@50:95: {map50_95:.4f}, Recall: {recall:.4f})")
            
            torch.save(self.model.model.state_dict(), last_path)
            logging.info(f"Saved last model state dict: {last_path} (Epoch: {epoch + 1}/{self.epochs})")
            print(f"Saved last model state dict: {last_path} (Epoch: {epoch + 1}/{self.epochs})")
            
            if score > self.best_score:
                self.best_score = score
                torch.save(self.model.model.state_dict(), best_path)
                logging.info(f"Updated best model state dict: {best_path} (Epoch: {epoch + 1}/{self.epochs}, Score: {score:.4f}, mAP@50:95: {map50_95:.4f}, Recall: {recall:.4f})")
                print(f"Updated best model state dict: {best_path} (Epoch: {epoch + 1}/{self.epochs}, Score: {score:.4f}, mAP@50:95: {map50_95:.4f}, Recall: {recall:.4f})")
            
            if recall > self.best_recall:
                self.best_recall = recall
                logging.info(f"New best recall: {recall:.4f} at epoch {epoch + 1}/{self.epochs}")
                print(f"New best recall: {recall:.4f} at epoch {epoch + 1}/{self.epochs}")
            
            try:
                dataset_dir = Path('/kaggle/working/checkpoint_dataset')
                os.makedirs(dataset_dir, exist_ok=True)
                shutil.copy(str(checkpoint_path), str(dataset_dir / "finalchechpoints.pt"))
                logging.info(f"Copied checkpoint to dataset directory: {dataset_dir}")
                print(f"Copied checkpoint to dataset directory: {dataset_dir}")
            except Exception as e:
                logging.error(f"Failed to copy checkpoint to dataset directory: {e}")
                print(f"ERROR: Failed to copy checkpoint to dataset directory: {e}")

        except Exception as e:
            logging.error(f"Failed to save checkpoint data to {checkpoint_path}: {e}")
            print(f"ERROR: Failed to save checkpoint data to {checkpoint_path}: {e}")

    def on_train_epoch_end(self, trainer):
        """Callback to run at the end of each training epoch."""
        print(f"Entering on_train_epoch_end for epoch {trainer.epoch + 1}/{self.epochs}")
        logging.info(f"Entering on_train_epoch_end for epoch {trainer.epoch + 1}/{self.epochs}")
        
        self.current_epoch = trainer.epoch
        
        if not hasattr(trainer, 'metrics'):
            logging.error("trainer.metrics not available. Cannot compute mAP and recall.")
            print("ERROR: trainer.metrics not available. Cannot compute mAP and recall.")
            map50_95 = 0.0
            recall = 0.0
        else:
            print(f"trainer.metrics contents: {trainer.metrics}")
            logging.info(f"trainer.metrics contents: {trainer.metrics}")
            map50_95 = trainer.metrics.get('metrics/mAP50-95(B)', 0.0)
            recall = trainer.metrics.get('metrics/recall(B)', 0.0)
            if map50_95 == 0.0 or recall == 0.0:
                logging.warning("mAP or recall is 0.0. Possible issue with validation data or metrics computation.")
                print("WARNING: mAP or recall is 0.0. Possible issue with validation data or metrics computation.")

        train_loss = getattr(trainer, 'loss', 0.0) if hasattr(trainer, 'loss') else 0.0
        val_loss = getattr(self.model.trainer, 'val_loss', 0.0) if self.model.trainer else 0.0
        loss_diff = abs(train_loss - val_loss)
        stability_penalty = min(max(0, (loss_diff - 0.1) * 0.05), 1.0)  # Cap penalty at 1.0

        score = 0.5 * map50_95 + 0.5 * recall - stability_penalty

        logging.info(f"Epoch {self.current_epoch + 1}/{self.epochs} completed. Score: {score:.4f}, mAP@50:95: {map50_95:.4f}, Recall: {recall:.4f}, Loss Diff: {loss_diff:.4f}")
        print(f"Epoch {self.current_epoch + 1}/{self.epochs} completed. Score: {score:.4f}, mAP@50:95: {map50_95:.4f}, Recall: {recall:.4f}, Loss Diff: {loss_diff:.4f}")

        self.save_checkpoint(self.current_epoch, map50_95, recall)

        if score > self.best_score:
            logging.info(f"Score improved from {self.best_score:.4f} to {score:.4f} at epoch {self.current_epoch + 1}/{self.epochs}.")
            print(f"Score improved from {self.best_score:.4f} to {score:.4f} at epoch {self.current_epoch + 1}/{self.epochs}.")
            self.best_score = score
            self.no_improve_epochs = 0
        else:
            self.no_improve_epochs += 1
            logging.info(f"Score did not improve ({score:.4f} vs best {self.best_score:.4f}). No improvement for {self.no_improve_epochs} epoch(s).")
            print(f"Score did not improve ({score:.4f} vs best {self.best_score:.4f}). No improvement for {self.no_improve_epochs} epoch(s).")

    def save_crash_checkpoint(self):
        """Save a checkpoint in case of a crash."""
        print("Entering save_crash_checkpoint")
        logging.info("Entering save_crash_checkpoint")
        try:
            checkpoint_dir = self.output_dir / 'yolov8l_vehicles_custom_weights' / 'check_points1'
            checkpoint_dir.mkdir(parents=True, exist_ok=True)
            crash_checkpoint_path = checkpoint_dir / 'crash_checkpoint.pt'
            torch.save({
                'epoch': self.current_epoch,
                'model_state_dict': self.model.model.state_dict(),
                'optimizer_state_dict': self.model.trainer.optimizer.state_dict() if hasattr(self.model, 'trainer') and self.model.trainer and hasattr(self.model.trainer, 'optimizer') else None,
                'timestamp': datetime.now().isoformat(),
            }, crash_checkpoint_path)
            print(f"Saved crash checkpoint: {crash_checkpoint_path}")
            logging.info(f"Saved crash checkpoint: {crash_checkpoint_path}")
        except Exception as e:
            logging.error(f"Failed to save crash checkpoint: {str(e)}")
            print(f"ERROR: Failed to save crash checkpoint: {str(e)}")

    def train(self):
        """Train YOLOv8l model starting from checkpoint for 17 epochs with targeted augmentation for weak classes."""
        print("Entering train")
        logging.info("Entering train")
        
        if self.model is None or self.hyp is None or self.weights_dir is None:
            logging.error("Training cannot start: Setup was not completed successfully.")
            print("ERROR: Training cannot start: Setup was not completed successfully.")
            return None
    
        try:
            checkpoint_path = str(self.checkpoint_input_path)
            if not Path(checkpoint_path).exists():
                logging.error(f"Checkpoint file not found at {checkpoint_path}. Cannot resume training.")
                print(f"ERROR: Checkpoint file not found at {checkpoint_path}. Cannot resume training.")
                raise RuntimeError(f"Checkpoint file not found at {checkpoint_path}")
    
            logging.info(f"Loading checkpoint: {checkpoint_path}")
            print(f"Loading checkpoint: {checkpoint_path}")
    
            # Load checkpoint to get model state and optimizer state
            checkpoint = torch.load(checkpoint_path, map_location=self.device)
            print(f"Checkpoint keys: {list(checkpoint.keys())}")
            logging.info(f"Checkpoint keys: {list(checkpoint.keys())}")
            print(f"Checkpoint epoch: {checkpoint.get('epoch', -1)}")
            logging.info(f"Checkpoint epoch: {checkpoint.get('epoch', -1)}")
    
            # Load model state
            self.model.model.load_state_dict(checkpoint['model_state_dict'])
    
            # Analyze checkpoint optimizer_state_dict
            print("Analyzing optimizer_state_dict from checkpoint...")
            logging.info("Analyzing optimizer_state_dict from checkpoint...")
            checkpoint_optimizer_state = checkpoint['optimizer_state_dict']
            for i, group in enumerate(checkpoint_optimizer_state['param_groups']):
                num_params = len(group['params'])
                print(f"Checkpoint parameter group {i}: {num_params} parameters, lr: {group['lr']}, weight_decay: {group['weight_decay']}")
                logging.info(f"Checkpoint parameter group {i}: {num_params} parameters, lr: {group['lr']}, weight_decay: {group['weight_decay']}")
    
            self.best_score = checkpoint.get('best_score_so_far', 0.0)
            self.best_recall = checkpoint.get('best_recall', 0.0)
            self.no_improve_epochs = 0  # Reset, but early stopping is disabled
    
            # Set epochs to 17 to start from epoch 1
            remaining_epochs = 17  # Train for 17 epochs starting from 1
            start_epoch = 1
    
            print(f"Starting training from epoch {start_epoch} for {remaining_epochs} epochs")
            logging.info(f"Starting training from epoch {start_epoch} for {remaining_epochs} epochs")
    
            # Override hyperparameters
            self.hyp['cls'] = 1.0
            self.hyp['dfl'] = 1.0
            self.hyp['patience'] = 1000
            self.hyp['mixup'] = 0.5
            self.hyp['mosaic'] = 0.8
            self.hyp['copy_paste'] = 0.95
            self.hyp['copy_paste_mode'] = 'flip'
            self.hyp['conf'] = 0.1
    
            print(f"Set cls weight to {self.hyp['cls']}, dfl weight to {self.hyp['dfl']}, mixup to {self.hyp['mixup']}, mosaic to {self.hyp['mosaic']}, copy_paste to {self.hyp['copy_paste']}, conf to {self.hyp['conf']}")
            logging.info(f"Set cls weight to {self.hyp['cls']}, dfl weight to {self.hyp['dfl']}, mixup to {self.hyp['mixup']}, mosaic to {self.hyp['mosaic']}, copy_paste to {self.hyp['copy_paste']}, conf to {self.hyp['conf']}")
    
            # Modified targeted copy-paste augmentation with albumentations
            def targeted_copy_paste(dataset, labels, prob=0.95):
                if np.random.rand() > prob or not isinstance(labels, dict):
                    return labels
                weak_classes = [2, 3]  # bus (2), motorcycle (3)
                new_labels = labels.copy()
                weak_images = [i for i, lbls in enumerate(dataset.labels) if any(lbl[0] in weak_classes for lbl in lbls)]
                if weak_images:
                    random_idx = np.random.choice(weak_images)
                    weak_labels = [lbl for lbl in dataset.labels[random_idx] if lbl[0] in weak_classes]
                    if weak_labels:
                        weak_labels = weak_labels[:3]
                        new_bboxes = np.array([lbl[1:] for lbl in weak_labels], dtype=np.float32)
                        new_cls = np.array([lbl[0] for lbl in weak_labels], dtype=np.float32)
                        
                        transform = A.Compose([
                            A.HorizontalFlip(p=0.5),
                            A.Rotate(limit=15, p=0.6),
                            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.4),
                            A.GaussNoise(p=0.2),
                            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.3),
                        ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.3))
                        
                        img_path = dataset.data[random_idx]['im_file']
                        img = cv2.imread(img_path)
                        if img is None:
                            logging.warning(f"Failed to load image {img_path} for targeted copy-paste")
                            return labels
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        
                        augmented = transform(image=img, bboxes=new_bboxes, class_labels=new_cls)
                        new_bboxes = augmented['bboxes']
                        new_cls = augmented['class_labels']
                        
                        if 'cls' in new_labels and 'bboxes' in new_labels:
                            new_labels['cls'] = np.concatenate([new_labels['cls'], new_cls])
                            new_labels['bboxes'] = np.concatenate([new_labels['bboxes'], new_bboxes])
                        else:
                            new_labels = {
                                'cls': new_cls,
                                'bboxes': new_bboxes,
                                'img_id': new_labels.get('img_id', 0),
                                'img_shape': new_labels.get('img_shape', (640, 640)),
                                'im_file': new_labels.get('im_file', '')
                            }
                return new_labels
    
            # Modified class-aware sampler with higher weight for weak classes
            def get_class_aware_sampler(dataset):
                weights = [5.0 if any(label[0] in [2, 3] for label in labels) else 1.0 for labels in dataset.labels]
                return WeightedRandomSampler(weights, len(weights), replacement=True)
    
            # Custom collate function to return a dictionary compatible with Ultralytics
            def custom_collate_fn(batch):
                images = []
                cls_list = []
                bboxes_list = []
                batch_idx = []
                im_files = []
                additional_keys = {}
                
                for idx, item in enumerate(batch):
                    if not isinstance(item, dict) or 'img' not in item:
                        logging.warning(f"Invalid batch item at index {idx}: {item}")
                        continue
                    
                    images.append(item['img'])
                    cls = item.get('cls', torch.tensor([], dtype=torch.float32))
                    bboxes = item.get('bboxes', torch.tensor([], dtype=torch.float32).reshape(0, 4))
                    
                    cls = torch.tensor(cls, dtype=torch.float32) if not isinstance(cls, torch.Tensor) else cls
                    bboxes = torch.tensor(bboxes, dtype=torch.float32) if not isinstance(bboxes, torch.Tensor) else bboxes
                    
                    if cls.numel() > 0 and bboxes.numel() > 0:
                        cls_list.append(cls)
                        bboxes_list.append(bboxes)
                        batch_idx.extend([idx] * cls.shape[0])
                    
                    im_file = item.get('im_file', '')
                    if isinstance(im_file, str):
                        im_files.append(im_file)
                    else:
                        logging.warning(f"Invalid im_file at index {idx}: {im_file}. Using empty string.")
                        im_files.append('')
                    
                    for key, value in item.items():
                        if key not in ['img', 'cls', 'bboxes', 'batch_idx', 'im_file']:
                            if key not in additional_keys:
                                additional_keys[key] = []
                            additional_keys[key].append(value)
                
                if not images:
                    logging.error("No valid images in batch")
                    raise ValueError("No valid images in batch")
                
                images = torch.stack(images, dim=0)
                
                cls_tensor = torch.cat(cls_list, dim=0) if cls_list else torch.tensor([], dtype=torch.float32)
                bboxes_tensor = torch.cat(bboxes_list, dim=0) if bboxes_list else torch.tensor([], dtype=torch.float32).reshape(0, 4)
                batch_idx_tensor = torch.tensor(batch_idx, dtype=torch.int64) if batch_idx else torch.tensor([], dtype=torch.int64)
                
                batch_dict = {
                    'img': images,
                    'cls': cls_tensor,
                    'bboxes': bboxes_tensor,
                    'batch_idx': batch_idx_tensor,
                    'im_file': im_files
                }
                
                for key, values in additional_keys.items():
                    if len(values) == len(images):
                        try:
                            if key == 'img_id':
                                values = [
                                    torch.tensor([v], dtype=torch.int64) if isinstance(v, (int, float)) or (isinstance(v, torch.Tensor) and v.numel() == 1)
                                    else v if isinstance(v, torch.Tensor)
                                    else torch.tensor([0], dtype=torch.int64)
                                    for v in values
                                ]
                                shapes = [v.shape for v in values]
                                if len(set(shapes)) == 1:
                                    batch_dict[key] = torch.stack(values, dim=0)
                                else:
                                    max_len = max(v.numel() for v in values)
                                    values = [
                                        torch.cat([v, torch.zeros(max_len - v.numel(), dtype=torch.int64)]) if v.numel() < max_len
                                        else v[:max_len]
                                        for v in values
                                    ]
                                    batch_dict[key] = torch.stack(values, dim=0)
                            elif key in ['img_shape', 'ori_shape', 'resized_shape']:
                                values = [
                                    torch.tensor(v, dtype=torch.int64) if isinstance(v, (tuple, list))
                                    else v if isinstance(v, torch.Tensor)
                                    else torch.tensor([640, 640], dtype=torch.int64)
                                    for v in values
                                ]
                                shapes = [v.shape for v in values]
                                if len(set(shapes)) == 1:
                                    batch_dict[key] = torch.stack(values, dim=0)
                                else:
                                    max_len = max(v.numel() for v in values)
                                    values = [
                                        torch.cat([v, torch.zeros(max_len - v.numel(), dtype=torch.int64)]) if v.numel() < max_len
                                        else v[:max_len]
                                        for v in values
                                    ]
                                    batch_dict[key] = torch.stack(values, dim=0)
                            else:
                                if all(isinstance(v, torch.Tensor) for v in values):
                                    shapes = [v.shape for v in values]
                                    if len(set(shapes)) == 1:
                                        batch_dict[key] = torch.stack(values, dim=0)
                                    else:
                                        max_len = max(v.numel() for v in values)
                                        values = [
                                            torch.cat([v, torch.zeros(max_len - v.numel(), dtype=v.dtype)]) if v.numel() < max_len
                                            else v[:max_len]
                                            for v in values
                                        ]
                                        batch_dict[key] = torch.stack(values, dim=0)
                                elif all(isinstance(v, (int, float)) for v in values):
                                    batch_dict[key] = torch.tensor(values, dtype=torch.int64 if all(isinstance(v, int) for v in values) else torch.float32)
                                else:
                                    logging.warning(f"Skipping additional key {key}: cannot convert to tensor")
                        except Exception as e:
                            logging.warning(f"Failed to process additional key {key}: {str(e)}. Skipping.")
                
                return batch_dict
    
            # Callback to modify DataLoader after initialization
            def on_pretrain_routine_end(trainer):
                print("Entering on_pretrain_routine_end callback")
                logging.info("Entering on_pretrain_routine_end callback")
                try:
                    train_dataset = trainer.train_loader.dataset
                    # Apply targeted copy-paste to dataset labels
                    train_dataset.labels = [targeted_copy_paste(train_dataset, labels, prob=self.hyp['copy_paste']) for labels in train_dataset.labels]
                    # Create a new DataLoader with WeightedRandomSampler
                    train_sampler = get_class_aware_sampler(train_dataset)
                    new_train_loader = torch.utils.data.DataLoader(
                        train_dataset,
                        batch_size=self.batch_size,
                        sampler=train_sampler,
                        num_workers=trainer.args.workers,
                        pin_memory=True,
                        collate_fn=custom_collate_fn,
                        shuffle=False  # Sampler handles shuffling
                    )
                    # Replace the trainer's train_loader
                    trainer.train_loader = new_train_loader
                    print("Successfully modified dataloader with class-aware sampling and targeted copy-paste")
                    logging.info("Successfully modified dataloader with class-aware sampling and targeted copy-paste")
                except Exception as e:
                    logging.error(f"Failed to modify dataloader: {str(e)}")
                    print(f"ERROR: Failed to modify dataloader: {str(e)}")
                    raise
    
            self.model.add_callback('on_pretrain_routine_end', on_pretrain_routine_end)
            self.model.add_callback('on_train_epoch_end', self.on_train_epoch_end)
    
            g0, g1, g2 = [], [], []
            g0_names, g1_names, g2_names = [], [], []
    
            all_params = [name for name, param in self.model.model.named_parameters() if param.requires_grad]
            print(f"Total trainable parameters: {len(all_params)}")
            logging.info(f"Total trainable parameters: {len(all_params)}")
            bn_weight_count = sum(1 for name in all_params if 'bn' in name and '.weight' in name)
            bn_bias_count = sum(1 for name in all_params if 'bn' in name and '.bias' in name)
            conv_bias_count = sum(1 for name in all_params if '.bias' in name and 'bn' not in name)
            print(f"Expected bn.weight count: {bn_weight_count}")
            print(f"Expected bn.bias count: {bn_bias_count}")
            print(f"Expected conv.bias count: {conv_bias_count}")
            logging.info(f"Expected bn.weight count: {bn_weight_count}")
            logging.info(f"Expected bn.bias count: {bn_bias_count}")
            logging.info(f"Expected conv.bias count: {conv_bias_count}")
    
            for name, param in self.model.model.named_parameters():
                if not param.requires_grad:
                    continue
                if 'bn' in name and '.bias' in name:
                    g2.append(param)
                    g2_names.append(name)
                elif 'bn' in name and '.weight' in name:
                    g0.append(param)
                    g0_names.append(name)
                elif '.bias' in name:
                    g0.append(param)
                    g0_names.append(name)
                else:
                    g1.append(param)
                    g1_names.append(name)
    
            print(f"Created parameter groups: g0={len(g0)}, g1={len(g1)}, g2={len(g2)}")
            logging.info(f"Created parameter groups: g0={len(g0)}, g1={len(g1)}, g2={len(g2)}")
    
            checkpoint_param_groups = checkpoint['optimizer_state_dict']['param_groups']
            param_groups = [
                {'params': g0, 'lr': checkpoint_param_groups[0]['lr'], 'weight_decay': checkpoint_param_groups[0]['weight_decay']},
                {'params': g1, 'lr': checkpoint_param_groups[1]['lr'], 'weight_decay': checkpoint_param_groups[1]['weight_decay']},
                {'params': g2, 'lr': checkpoint_param_groups[2]['lr'], 'weight_decay': checkpoint_param_groups[2]['weight_decay']}
            ]
    
            optimizer = torch.optim.AdamW(
                param_groups,
                betas=(0.937, 0.999)
            )
    
            print(f"Number of parameter groups in new optimizer: {len(optimizer.param_groups)}")
            logging.info(f"Number of parameter groups in new optimizer: {len(optimizer.param_groups)}")
            print(f"Parameters per group in new optimizer: {[len(group['params']) for group in optimizer.param_groups]}")
            logging.info(f"Parameters per group in new optimizer: {[len(group['params']) for group in optimizer.param_groups]}")
            print(f"Group 0 (bn weights + conv biases): {len(g0_names)} parameters")
            logging.info(f"Group 0 (bn weights + conv biases): {len(g0_names)} parameters")
            print(f"Group 1 (other weights): {len(g1_names)} parameters")
            logging.info(f"Group 1 (other weights): {len(g1_names)} parameters")
            print(f"Group 2 (bn biases): {len(g2_names)} parameters")
            logging.info(f"Group 2 (bn biases): {len(g2_names)} parameters")
            print(f"First 5 Group 0 parameters: {g0_names[:5]}")
            logging.info(f"First 5 Group 0 parameters: {g0_names[:5]}")
            print(f"First 5 Group 1 parameters: {g1_names[:5]}")
            logging.info(f"First 5 Group 1 parameters: {g1_names[:5]}")
            print(f"First 5 Group 2 parameters: {g2_names[:5]}")
            logging.info(f"First 5 Group 2 parameters: {g2_names[:5]}")
    
            try:
                optimizer.load_state_dict(checkpoint_optimizer_state)
                logging.info(f"Loaded optimizer state from checkpoint")
                print(f"Loaded optimizer state from checkpoint")
            except Exception as e:
                logging.error(f"Failed to load optimizer state: {str(e)}")
                print(f"ERROR: Failed to load optimizer state: {str(e)}")
                raise RuntimeError(f"Failed to load optimizer state: {str(e)}")
    
            checkpoint_lr = checkpoint_optimizer_state['param_groups'][0]['lr']
            for group in optimizer.param_groups:
                group['lr'] = checkpoint_lr
                print(f"Set lr for parameter group to {group['lr']}")
                logging.info(f"Set lr for parameter group to {group['lr']}")
    
            self.model.trainer = type('Trainer', (), {})()
            self.model.trainer.optimizer = optimizer
    
            # Add a dummy reset method to DataLoader to avoid AttributeError
            def dummy_reset(self):
                pass
            torch.utils.data.DataLoader.reset = dummy_reset
    
            results = self.model.train(
                data=str(self.data_yaml),
                epochs=remaining_epochs,
                batch=self.batch_size,
                imgsz=self.imgsz,
                device=self.device,
                project=str(self.output_dir / 'yolov8l_vehicles_custom_weights'),
                name='check_points1',
                exist_ok=True,
                save=True,
                save_period=self.save_period,
                cache=False,
                plots=True,
                verbose=True,
                pretrained=False,
                resume=False,
                **self.hyp
            )
    
            logging.info(f"Training loop finished. Best Score: {self.best_score:.4f}, Best Recall: {self.best_recall:.4f}")
            print(f"Training loop finished. Best Score: {self.best_score:.4f}, Best Recall: {self.best_recall:.4f}")
            return results
    
        except Exception as error:
            logging.error(f"Training failed: {str(error)}")
            print(f"ERROR: Training failed: {str(error)}")
            self.save_crash_checkpoint()
            torch.cuda.empty_cache()
            logging.info("Cleared CUDA cache.")
            print("Cleared CUDA cache.")
            raise

    def evaluate(self, split='val'):
        """Evaluate the model on the specified split (val or test)."""
        print(f"Starting evaluation on '{split}' split...")
        logging.info(f"Starting evaluation on '{split}' split...")
        try:
            metrics = self.model.val(data=str(self.data_yaml), split=split, plots=True, conf=self.hyp.get('conf', 0.25))
            recall = metrics.box.mr() if hasattr(metrics.box, 'mr') else metrics.box.r.mean()
            logging.info(f"Evaluation Metrics ({split} split): mAP50-95: {metrics.box.map:.4f}, Recall: {recall:.4f}")
            print(f"Evaluation Metrics ({split} split): mAP50-95: {metrics.box.map:.4f}, Recall: {recall:.4f}")
            return metrics
        except Exception as e:
            logging.error(f"Evaluation on '{split}' split failed: {str(e)}")
            print(f"ERROR: Evaluation on '{split}' split failed: {str(e)}")
            raise

    def save_final_model(self, filename="mostafafinal.pt"):
        """Save final trained model using model.save()."""
        print("Entering save_final_model")
        logging.info("Entering save_final_model")
        
        if self.model is None:
            logging.error("No model available to save.")
            print("ERROR: No model available to save.")
            return
        if not self.weights_dir:
            logging.error("Weights directory not set. Cannot save final model.")
            print("ERROR: Weights directory not set. Cannot save final model.")
            return
            
        try:
            checkpoint_dir = self.weights_dir / 'check_points1'
            os.makedirs(checkpoint_dir, exist_ok=True)
            final_path = checkpoint_dir / filename
            self.model.save(str(final_path))
            logging.info(f"Final model saved using model.save() to {final_path}")
            print(f"Final model saved using model.save() to {final_path}")
            self.clean_old_checkpoints()
        except Exception as e:
            logging.error(f"Failed to save final model to {final_path}: {e}")
            print(f"ERROR: Failed to save final model to {final_path}: {e}")
            raise

    def run(self):
        """Run full training and evaluation pipeline."""
        start_time = datetime.now()
        logging.info(f"Starting YOLOv8 Training Pipeline at {start_time}")
        print(f"Starting YOLOv8 Training Pipeline at {start_time}")
        
        if not self.setup():
            logging.critical("Pipeline setup failed. Exiting.")
            print("CRITICAL: Pipeline setup failed. Exiting.")
            return

        train_results = self.train()
        
        if train_results is not None:
            logging.info("Training process completed.")
            print("Training process completed.")
            self.evaluate(split='val')
            self.evaluate(split='test')
            self.save_final_model()
        else:
            logging.error("Training process failed or was skipped.")
            print("ERROR: Training process failed or was skipped.")

        end_time = datetime.now()
        logging.info(f"YOLOv8 Training Pipeline finished at {end_time}")
        print(f"YOLOv8 Training Pipeline finished at {end_time}")
        logging.info(f"Total execution time: {end_time - start_time}")
        print(f"Total execution time: {end_time - start_time}")

if __name__ == "__main__":
    trainer = YOLOv8Trainer()
    trainer.run()

In [8]:
import logging
import os
from ultralytics import YOLO
import torch
import yaml
import numpy as np

class YOLOv8Trainer:
    def __init__(self, model_path="/kaggle/working/runs/yolov8l_vehicles_custom_weights/check_points1/weights/best.pt", 
                 data_yaml="/kaggle/working/detection-cars-cleaned/data.yaml"):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        logging.info(f"Using device: {self.device}")
        self.model_path = model_path
        self.data_yaml = data_yaml
        self.hyp = self.load_hyp_yaml("/kaggle/working/aug_hyp.yaml")
        self.model = self.load_model()

    def load_hyp_yaml(self, hyp_path):
        logging.info(f"Loading hyperparameters from {hyp_path}")
        with open(hyp_path, 'r') as f:
            hyp = yaml.safe_load(f)
        return hyp

    def load_model(self):
        logging.info(f"Loading model from {self.model_path}")
        model = YOLO(self.model_path)
        model.to(self.device)
        return model

    def evaluate(self, split='val'):
        logging.info(f"Starting evaluation on '{split}' split...")
        try:
            metrics = self.model.val(data=str(self.data_yaml), split=split, plots=True, conf=self.hyp.get('conf', 0.25))
            recall = metrics.box.mr if hasattr(metrics.box, 'mr') else np.mean(metrics.box.r)
            logging.info(f"Evaluation Metrics ({split} split): mAP50-95: {metrics.box.map:.4f}, Recall: {recall:.4f}")
            print(f"Evaluation Metrics ({split} split): mAP50-95: {metrics.box.map:.4f}, Recall: {recall:.4f}")
            return metrics
        except Exception as e:
            logging.error(f"Evaluation on '{split}' split failed: {str(e)}")
            raise

    def run(self):
        logging.info("Starting YOLOv8 Evaluation Pipeline")
        try:
            # Skip training and go directly to evaluation
            self.evaluate(split='val')
            self.evaluate(split='test')
        except Exception as e:
            logging.error(f"Pipeline failed: {str(e)}")
            raise

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    trainer = YOLOv8Trainer()
    trainer.run()

2025-05-03 09:47:36,487 - INFO - Using device: cuda:0
2025-05-03 09:47:36,488 - INFO - Loading hyperparameters from /kaggle/working/aug_hyp.yaml
2025-05-03 09:47:36,492 - INFO - Loading model from /kaggle/working/runs/yolov8l_vehicles_custom_weights/check_points1/weights/best.pt
2025-05-03 09:47:36,835 - INFO - Starting YOLOv8 Evaluation Pipeline
2025-05-03 09:47:36,836 - INFO - Starting evaluation on 'val' split...


Model summary (fused): 112 layers, 43,609,692 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1400.5±551.7 MB/s, size: 69.1 KB)


val: Scanning /kaggle/working/detection-cars-cleaned/val/labels.cache... 6297 images, 1 backgrounds, 0 corrupt: 100%|██████████| 6297/6297 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 394/394 [01:49<00:00,  3.59it/s]


                   all       6297       8901      0.802      0.786      0.835      0.636
                   car       5417       7137       0.94      0.968      0.982      0.876
                 truck        229        290      0.746      0.745      0.789      0.563
                   bus        611       1237      0.836      0.816      0.866      0.609
            motorcycle        198        237      0.688      0.616      0.704      0.496


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


...

                   all       6298       9180      0.839      0.775      0.849      0.647
                   car       5424       7406      0.941      0.962      0.981      0.873
                 truck        213        290       0.76      0.659      0.764      0.568
                   bus        644       1266      0.873      0.775       0.86      0.606
            motorcycle        191        218      0.781      0.706      0.793      0.539


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 14.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val2


2025-05-03 09:51:35,980 - INFO - Evaluation Metrics (test split): mAP50-95: 0.6468, Recall: 0.7755


Evaluation Metrics (test split): mAP50-95: 0.6468, Recall: 0.7755
